In [1]:
%cd ../
%matplotlib inline

/home/eli/AnacondaProjects/HTFATorch


In [2]:
import collections
import logging
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils
import glob
import os

import numpy as np
from sklearn.model_selection import LeaveOneOut

In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
affvids_dir = '/home/eli/Documents/AffVids/motion_corrected/'

task_log_csvs = 'editedlogfiles'

TASK_ONSET_DELAY = 3

AFFVID_FILENAME_TEMPLATE = 'sub%02d_run%02d_mc_MNI_masked.nii.gz'
MASK_FILENAME_TEMPLATE = 'sub-%02d_task-AffVids_run-%02d_bold_space-MNI152NLin2009cAsym_brainmask.nii.gz'

AFFVIDS_FILE_TEMPLATE = 'data/affvids2018_%d.db'

def affvid_filename(subject, run):
    return affvids_dir + (AFFVID_FILENAME_TEMPLATE % (subject, run))

def mask_filename(subject, run):
    return (affvids_dir + 'masks_norest/subject-%02d/func/' + MASK_FILENAME_TEMPLATE) % (subject, subject, run)

class TaskElement:
    def __init__(self, task, start, end, run, fear_rating=None):
        def round_off_time(t):
            if t is not None:
                if task != 'rest':
                    return round(t + TASK_ONSET_DELAY)
                else:
                    return round(t)
            else:
                return None
        self.task = task
        self.start_time = round_off_time(start)
        self.end_time = round_off_time(end)
        self.run = run
        self.fear_rating = fear_rating

def parse_task_lines(lines):
    for (i, line) in enumerate(lines):
        cols = line.split(' ')
        task = cols[0]
        start_time = float(cols[9])
        end_time = float(cols[10])
        run = int(cols[1])
        fear_rating = float(cols[11])
        yield TaskElement(task, start_time, end_time, run, fear_rating)

def rest_tasks(tasks):
    yield TaskElement('rest', 0, tasks[0].start_time - 1, tasks[0].run)
    for i in range(1, len(tasks)):
        rest_start = tasks[i-1].end_time + 1
        rest_end = tasks[i].start_time - 1
        if tasks[i].run == tasks[i-1].run:
            yield TaskElement('rest', rest_start, rest_end, tasks[i].run)
        else:
            yield TaskElement('rest', rest_start, None, tasks[i-1].run)
            yield TaskElement('rest', 0, rest_end, tasks[i].run)
    yield TaskElement('rest', tasks[-1].end_time + 1, None, tasks[-1].run)

bad_runs = collections.defaultdict(lambda: [])
bad_runs[3] = [1, 2, 3]
bad_runs[7] = [3]
bad_runs[14] = [3]
bad_runs[22] = [1, 2, 3]
bad_runs[23] = [1]
bad_runs[24] = [1, 2, 3]
bad_runs[26] = [3]

def read_tasks(task_csv):
    def sentinel(f):
        return f if f is not None else 0.0
    with open(task_csv, 'r') as task_csv_file:
        header = task_csv_file.readline().split(' ')
        subject = int(header[4])
        logging.info('Subject %d', subject)
        task_lines = list(parse_task_lines(task_csv_file.readlines()))
        task_lines += list(rest_tasks(task_lines))
        task_lines = sorted(task_lines, key=lambda t: sentinel(t.start_time))
        for (i, task) in enumerate(task_lines):
            if task.run in bad_runs[subject]:
                logging.info('Excluding block %d %s of run %d for motion', i,
                             task.task, task.run)
                continue
            logging.info('Block %d %s of run %d started at %f, ended at %f', i,
                         task.task, task.run, sentinel(task.start_time), sentinel(task.end_time))
            result = niidb.FMriActivationBlock(zscore=True)
            result.subject = subject
            result.task = task.task
            result.run = task.run
            result.block = i
            result.start_time = task.start_time
            result.end_time = task.end_time
            result.individual_differences = {'fear_rating': task.fear_rating}
            yield result

In [5]:
RUNS = set()

for task_csv in utils.sorted_glob(affvids_dir + task_log_csvs + '/*.txt'):
    for block in read_tasks(task_csv):
        RUNS = RUNS | {block.run}

10/07/2020 10:15:37 Subject 10
10/07/2020 10:15:37 Block 0 rest of run 1 started at 0.000000, ended at 24.000000
10/07/2020 10:15:37 Block 1 rest of run 2 started at 0.000000, ended at 22.000000
10/07/2020 10:15:37 Block 2 rest of run 3 started at 0.000000, ended at 22.000000
10/07/2020 10:15:37 Block 3 spider_low_3.m4v of run 2 started at 23.000000, ended at 44.000000
10/07/2020 10:15:37 Block 4 heights_low_6.m4v of run 3 started at 23.000000, ended at 41.000000
10/07/2020 10:15:37 Block 5 spider_high_5.m4v of run 1 started at 25.000000, ended at 46.000000
10/07/2020 10:15:37 Block 6 rest of run 3 started at 42.000000, ended at 63.000000
10/07/2020 10:15:37 Block 7 rest of run 2 started at 45.000000, ended at 68.000000
10/07/2020 10:15:37 Block 8 rest of run 1 started at 47.000000, ended at 69.000000
10/07/2020 10:15:37 Block 9 heights_high_2.mov of run 3 started at 64.000000, ended at 85.000000
10/07/2020 10:15:37 Block 10 heights_high_1.m4v of run 2 started at 69.000000, ended at 89

In [6]:
OVERRIDE = True

def write_db_file(k, train_runs):
    affvids_file = AFFVIDS_FILE_TEMPLATE % k
    if not os.path.exists(affvids_file) or OVERRIDE:
        affvids_db = niidb.FMriActivationsDb(affvids_file, mask='/home/eli/Documents/AffVids/wholebrain2.nii.gz',
                                             smooth=6)

        for task_csv in utils.sorted_glob(affvids_dir + task_log_csvs + '/*.txt'):
            for block in read_tasks(task_csv):
                block.filename = affvid_filename(block.subject, block.run)
                block.mask = mask_filename(block.subject, block.run)
                affvids_db.upsert(block)

In [7]:
cv = LeaveOneOut()
for k, (train_runs, val_runs) in enumerate(cv.split(np.zeros(len(RUNS)), list(RUNS))):
    write_db_file(k, train_runs)

10/07/2020 10:15:38 Subject 10
10/07/2020 10:15:38 Block 0 rest of run 1 started at 0.000000, ended at 24.000000
10/07/2020 10:15:38 Block 1 rest of run 2 started at 0.000000, ended at 22.000000
10/07/2020 10:15:38 Block 2 rest of run 3 started at 0.000000, ended at 22.000000
10/07/2020 10:15:38 Block 3 spider_low_3.m4v of run 2 started at 23.000000, ended at 44.000000
10/07/2020 10:15:38 Block 4 heights_low_6.m4v of run 3 started at 23.000000, ended at 41.000000
10/07/2020 10:15:38 Block 5 spider_high_5.m4v of run 1 started at 25.000000, ended at 46.000000
10/07/2020 10:15:38 Block 6 rest of run 3 started at 42.000000, ended at 63.000000
10/07/2020 10:15:38 Block 7 rest of run 2 started at 45.000000, ended at 68.000000
10/07/2020 10:15:38 Block 8 rest of run 1 started at 47.000000, ended at 69.000000
10/07/2020 10:15:38 Block 9 heights_high_2.mov of run 3 started at 64.000000, ended at 85.000000
10/07/2020 10:15:38 Block 10 heights_high_1.m4v of run 2 started at 69.000000, ended at 89

In [8]:
logging.info('Finished building NiiDb out of AffVids dataset in %s', affvids_dir)

10/07/2020 10:22:00 Finished building NiiDb out of AffVids dataset in /home/eli/Documents/AffVids/motion_corrected/
